In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 🎬 Sports Video Analysis - Google Colab\n",
    "\n",
    "Este notebook permite testar seus modelos YOLO e análises de vídeo de futebol no Google Colab.\n",
    "\n",
    "## 📋 Instruções:\n",
    "1. Execute as células de cima para baixo (de forma sequencial)\n",
    "2. Quando pedir para fazer upload, selecione seu vídeo\n",
    "3. Escolha o modo de análise desejado\n",
    "4. Baixe o vídeo processado ao final"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1️⃣ Instalar Dependências"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "!pip install opencv-python supervision numpy matplotlib pillow ultralytics tqdm -q\n",
    "print(\"✅ Dependências instaladas com sucesso!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2️⃣ Clonar Repositório (ou fazer upload dos arquivos)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# OPÇÃO A: Clonar do GitHub (descomente a linha abaixo)\n",
    "# !git clone https://github.com/seu-usuario/sports.git /content/sports-main\n",
    "\n",
    "# OPÇÃO B: Se você vai fazer upload manual, use o painel de Files no lado esquerdo\n",
    "# Para isso, clique no ícone de pasta no lado esquerdo e faça upload da pasta 'sports-main'\n",
    "\n",
    "import os\n",
    "if os.path.exists('/content/sports-main'):\n",
    "    print(\"✅ Repositório encontrado!\")\n",
    "    !ls -la /content/sports-main/\n",
    "else:\n",
    "    print(\"⚠️  Repositório não encontrado. Faça upload da pasta 'sports-main' usando o painel Files no lado esquerdo.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3️⃣ Configurar Python Path e Importar Módulos"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import sys\n",
    "import os\n",
    "\n",
    "# Adicionar ao path\n",
    "sys.path.insert(0, '/content/sports-main')\n",
    "\n",
    "# Importar módulos do projeto\n",
    "from sports.configs.soccer import SoccerPitchConfiguration\n",
    "from sports.annotators.soccer import draw_pitch, draw_points_on_pitch\n",
    "from sports.common.ball import BallTracker, BallAnnotator\n",
    "from sports.common.team import TeamClassifier\n",
    "from sports.common.view import ViewTransformer\n",
    "\n",
    "# Importar dependências\n",
    "import cv2\n",
    "import numpy as np\n",
    "import supervision as sv\n",
    "from ultralytics import YOLO\n",
    "from tqdm import tqdm\n",
    "from enum import Enum\n",
    "from collections import deque\n",
    "import matplotlib.pyplot as plt\n",
    "from IPython.display import Video, HTML\n",
    "\n",
    "print(\"✅ Todos os módulos importados com sucesso!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4️⃣ Carregar Modelos YOLO"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Definir caminhos dos modelos\n",
    "MODEL_DIR = '/content/sports-main/examples/soccer/data'\n",
    "\n",
    "PLAYER_DETECTION_MODEL_PATH = os.path.join(MODEL_DIR, 'football-player-detection.pt')\n",
    "PITCH_DETECTION_MODEL_PATH = os.path.join(MODEL_DIR, 'football-pitch-detection.pt')\n",
    "BALL_DETECTION_MODEL_PATH = os.path.join(MODEL_DIR, 'football-ball-detection.pt')\n",
    "\n",
    "# Verificar se os modelos existem\n",
    "print(\"\\n📁 Verificando modelos:\")\n",
    "print(f\"Player detection: {'✅' if os.path.exists(PLAYER_DETECTION_MODEL_PATH) else '❌'} {PLAYER_DETECTION_MODEL_PATH}\")\n",
    "print(f\"Pitch detection: {'✅' if os.path.exists(PITCH_DETECTION_MODEL_PATH) else '❌'} {PITCH_DETECTION_MODEL_PATH}\")\n",
    "print(f\"Ball detection: {'✅' if os.path.exists(BALL_DETECTION_MODEL_PATH) else '❌'} {BALL_DETECTION_MODEL_PATH}\")\n",
    "\n",
    "# Carregar modelos\n",
    "print(\"\\n⏳ Carregando modelos... (pode levar alguns minutos)\")\n",
    "player_detector = YOLO(PLAYER_DETECTION_MODEL_PATH)\n",
    "pitch_detector = YOLO(PITCH_DETECTION_MODEL_PATH)\n",
    "ball_detector = YOLO(BALL_DETECTION_MODEL_PATH)\n",
    "\n",
    "print(\"✅ Modelos carregados com sucesso!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5️⃣ Fazer Upload do Vídeo"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "from google.colab import files\n",
    "\n",
    "print(\"\\n📹 Selecione seu arquivo de vídeo...\")\n",
    "print(\"Formatos suportados: .mp4, .avi, .mov, .mkv\")\n",
    "\n",
    "uploaded = files.upload()\n",
    "video_path = list(uploaded.keys())[0]\n",
    "\n",
    "print(f\"\\n✅ Vídeo enviado: {video_path}\")\n",
    "print(f\"Tamanho: {os.path.getsize(video_path) / (1024**2):.2f} MB\")\n",
    "\n",
    "# Verificar info do vídeo\n",
    "cap = cv2.VideoCapture(video_path)\n",
    "fps = cap.get(cv2.CAP_PROP_FPS)\n",
    "frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))\n",
    "width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))\n",
    "height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))\n",
    "cap.release()\n",
    "\n",
    "print(f\"\\n📊 Informações do vídeo:\")\n",
    "print(f\"  Resolução: {width}x{height}\")\n",
    "print(f\"  FPS: {fps}\")\n",
    "print(f\"  Frames: {frame_count}\")\n",
    "print(f\"  Duração: {frame_count/fps:.2f}s\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6️⃣ Selecionar Modo de Análise"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Definir modo de análise\n",
    "class Mode(Enum):\n",
    "    PLAYER_DETECTION = \"player_detection\"\n",
    "    BALL_DETECTION = \"ball_detection\"\n",
    "    PITCH_DETECTION = \"pitch_detection\"\n",
    "    PLAYER_TRACKING = \"player_tracking\"\n",
    "    COMBINED_ANALYSIS = \"combined_analysis\"\n",
    "\n",
    "# Escolha o modo (mude aqui para testar diferentes modos)\n",
    "selected_mode = Mode.PLAYER_DETECTION  # Altere para: BALL_DETECTION, PITCH_DETECTION, COMBINED_ANALYSIS, etc.\n",
    "\n",
    "print(f\"\\n🎯 Modo selecionado: {selected_mode.value.upper()}\")\n",
    "print(\"\\n📝 Modos disponíveis:\")\n",
    "for mode in Mode:\n",
    "    print(f\"  - {mode.value}\")\n",
    "print(f\"\\n💡 Para mudar de modo, edite a linha 'selected_mode = Mode.PLAYER_DETECTION' acima\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7️⃣ Processar Vídeo"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Configurações\n",
    "STRIDE = 30  # Processar a cada N frames (aumentar para mais rápido, diminuir para mais preciso)\n",
    "CONFIDENCE = 0.5  # Confiança mínima para detecções\n",
    "\n",
    "output_path = '/content/output_video.mp4'\n",
    "os.makedirs(os.path.dirname(output_path) or '.', exist_ok=True)\n",
    "\n",
    "# Abrir vídeo de entrada\n",
    "cap = cv2.VideoCapture(video_path)\n",
    "fps = cap.get(cv2.CAP_PROP_FPS)\n",
    "frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))\n",
    "width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))\n",
    "height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))\n",
    "\n",
    "# Criar writer para vídeo de saída\n",
    "fourcc = cv2.VideoWriter_fourcc(*'mp4v')\n",
    "out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))\n",
    "\n",
    "print(f\"\\n⏳ Processando vídeo...\")\n",
    "print(f\"  Modo: {selected_mode.value}\")\n",
    "print(f\"  Stride: {STRIDE} frames\")\n",
    "print(f\"  Total de frames: {frame_count}\")\n",
    "\n",
    "frame_id = 0\n",
    "pbar = tqdm(total=frame_count, desc=\"Processando\")\n",
    "\n",
    "while True:\n",
    "    ret, frame = cap.read()\n",
    "    if not ret:\n",
    "        break\n",
    "    \n",
    "    # Processar a cada N frames\n",
    "    if frame_id % STRIDE == 0:\n",
    "        if selected_mode == Mode.PLAYER_DETECTION:\n",
    "            # Detectar jogadores\n",
    "            results = player_detector.predict(frame, conf=CONFIDENCE, verbose=False)\n",
    "            detections = sv.Detections.from_ultralytics(results[0])\n",
    "            frame = sv.BoxAnnotator().annotate(scene=frame, detections=detections)\n",
    "            frame = sv.LabelAnnotator().annotate(scene=frame, detections=detections)\n",
    "        \n",
    "        elif selected_mode == Mode.BALL_DETECTION:\n",
    "            # Detectar bola\n",
    "            results = ball_detector.predict(frame, conf=CONFIDENCE, verbose=False)\n",
    "            detections = sv.Detections.from_ultralytics(results[0])\n",
    "            frame = sv.CircleAnnotator(radius=10).annotate(scene=frame, detections=detections)\n",
    "        \n",
    "        elif selected_mode == Mode.PITCH_DETECTION:\n",
    "            # Detectar campo\n",
    "            results = pitch_detector.predict(frame, conf=CONFIDENCE, verbose=False)\n",
    "            detections = sv.Detections.from_ultralytics(results[0])\n",
    "            frame = sv.BoxAnnotator().annotate(scene=frame, detections=detections)\n",
    "        \n",
    "        elif selected_mode == Mode.COMBINED_ANALYSIS:\n",
    "            # Análise combinada: Jogadores + Bola\n",
    "            player_results = player_detector.predict(frame, conf=CONFIDENCE, verbose=False)\n",
    "            ball_results = ball_detector.predict(frame, conf=CONFIDENCE, verbose=False)\n",
    "            \n",
    "            player_detections = sv.Detections.from_ultralytics(player_results[0])\n",
    "            ball_detections = sv.Detections.from_ultralytics(ball_results[0])\n",
    "            \n",
    "            # Anotar jogadores\n",
    "            frame = sv.BoxAnnotator(color=sv.Color.BLUE).annotate(\n",
    "                scene=frame, detections=player_detections\n",
    "            )\n",
    "            # Anotar bola\n",
    "            frame = sv.CircleAnnotator(radius=8, color=sv.Color.RED).annotate(\n",
    "                scene=frame, detections=ball_detections\n",
    "            )\n",
    "    \n",
    "    # Escrever frame no vídeo de saída\n",
    "    out.write(frame)\n",
    "    frame_id += 1\n",
    "    pbar.update(1)\n",
    "\n",
    "cap.release()\n",
    "out.release()\n",
    "pbar.close()\n",
    "\n",
    "print(f\"\\n✅ Vídeo processado com sucesso!\")\n",
    "print(f\"📁 Salvo em: {output_path}\")\n",
    "print(f\"📊 Tamanho: {os.path.getsize(output_path) / (1024**2):.2f} MB\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 8️⃣ Visualizar Resultado"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Exibir vídeo processado\n",
    "print(\"\\n▶️  Exibindo vídeo processado...\\n\")\n",
    "Video(output_path, width=800)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 9️⃣ Baixar Vídeo Processado"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"\\n📥 Preparando para download...\\n\")\n",
    "files.download(output_path)\n",
    "print(f\"\\n✅ Download iniciado!\")\n",
    "print(f\"📁 Arquivo: output_video.mp4\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🔧 Opções Avançadas"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Ajustar Parâmetros e Reprocessar"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Você pode copiar a célula 7 (Processar Vídeo) aqui e ajustar os parâmetros:\n",
    "# - STRIDE: aumentar para processar menos frames (mais rápido)\n",
    "# - CONFIDENCE: aumentar para menos detecções falsas\n",
    "# - selected_mode: trocar o modo de análise\n",
    "\n",
    "print(\"\\n💡 Dicas para ajustar:\")\n",
    "print(\"\\n1. Aumentar STRIDE (ex: 60) para processar mais rápido\")\n",
    "print(\"2. Aumentar CONFIDENCE (ex: 0.7) para reduzir falsos positivos\")\n",
    "print(\"3. Mudar selected_mode para testar diferentes modos\")\n",
    "print(\"4. Copie a célula 7 inteira, ajuste os parâmetros e execute novamente\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Ver Primeiros Frames do Resultado"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Extrair e exibir primeiros frames\n",
    "cap = cv2.VideoCapture(output_path)\n",
    "frames_to_show = 3\n",
    "\n",
    "fig, axes = plt.subplots(1, frames_to_show, figsize=(15, 5))\n",
    "\n",
    "for i in range(frames_to_show):\n",
    "    ret, frame = cap.read()\n",
    "    if ret:\n",
    "        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)\n",
    "        axes[i].imshow(frame_rgb)\n",
    "        axes[i].set_title(f\"Frame {i*STRIDE}\")\n",
    "        axes[i].axis('off')\n",
    "\n",
    "cap.release()\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "print(f\"\\n✅ Primeiros frames do resultado exibidos!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📚 Guia de Uso Rápido\n",
    "\n",
    "### Primeiro uso:\n",
    "1. Execute **célula 1** → Instala dependências\n",
    "2. Execute **célula 2** → Faz upload ou clona repositório\n",
    "3. Execute **célula 3** → Carrega módulos\n",
    "4. Execute **célula 4** → Carrega modelos YOLO\n",
    "5. Execute **célula 5** → Faz upload do vídeo\n",
    "6. Execute **célula 6** → Seleciona modo (edite se quiser mudar)\n",
    "7. Execute **célula 7** → Processa vídeo (⏳ pode demorar)\n",
    "8. Execute **célula 8** → Visualiza resultado\n",
    "9. Execute **célula 9** → Baixa vídeo processado\n",
    "\n",
    "### Próximos usos:\n",
    "- Pule as células 1-4 (já estão instaladas/carregadas)\n",
    "- Faça upload de novo vídeo na célula 5\n",
    "- Ajuste modo/parâmetros e reprocesse\n",
    "\n",
    "### 💡 Dicas de Velocidade:\n",
    "- Ative **GPU** no Colab: Runtime → Change runtime type → GPU\n",
    "- Aumente `STRIDE` (ex: 60) para processar menos frames\n",
    "- Use vídeos menores para testes rápidos\n",
    "- Aumente `CONFIDENCE` para menos detecções"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.10.12"
  },
  "accelerator": "GPU"
 },
 "nbformat": 4,
 "nbformat_minor": 4
}